### Notebook for running pipeline in Colab

If you're running on colab: run these cells first!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/BEP/git/GPT-DT
!git status
!git pull
# Choose branch name here:
!git checkout evaluation-loop

In [ ]:
%pip install -r requirements-colab.txt

In [ ]:
# CLICK RESTART RUNTIME AND THEN RUN THIS CELL
%cd /content/drive/MyDrive/BEP/git/GPT-DT

### Main Code

In [ ]:
%matplotlib inline
from main import *

In [ ]:
image = "vraag 4.jpg"
prompt, response, car_speed = run_program(image)

In [ ]:
response

In [5]:
truth = pd.read_csv("ground-truth.csv")
results = truth.copy(deep = True)
results["Answer(word)"] = None
results["Answer(letter)"] = None
results["Speed"] = None

In [10]:
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# len(truth.index)

for row in range(len(truth.index)):
    tru_row = truth.loc[row]
    res_row = results.loc[row]

    image = tru_row["Filename"]
    prompt, response, car_speed = run_program(image)
    resp_char = response.strip(' \n\t')[0]

    if resp_char == 'A':
        resp_word = "Brake"
    elif resp_char == 'B':
        resp_word = "Release Accelerator"
    elif resp_char == 'C':
        resp_word = "Nothing"
    else:
        resp_word = "unknown"

    res_row["Answer(letter)"] = resp_char
    res_row["Speed"] = car_speed
    res_row["Answer(word)"] = resp_word

In [13]:
confu = metrics.confusion_matrix(truth.head(41)[["Answer(letter)"]], results.head(41)[["Answer(letter)"]])
score = metrics.accuracy_score(truth.head(41)[["Answer(letter)"]], results.head(41)[["Answer(letter)"]])


In [ ]:
sns.heatmap(confu, annot=True, cmap='magma')
print(score)
plt.show()

In [ ]:
sns.heatmap(confu, annot=True, cmap='magma')
print(score)
plt.show()